<a href="https://colab.research.google.com/github/andriy8800555355/LSTMMIDIAI/blob/main/LSTMMidiAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Завантажити необхідні лібки
!pip install numpy tensorflow music21

In [ ]:
#@title Навчити модель на ваших мідішках
import numpy as np
import tensorflow as tf
from tensorflow import keras
from music21 import converter, instrument, note, chord
import glob
import pickle
import os
from google.colab import drive
from google.colab import files
import matplotlib.pyplot as plt

def is_drive_mounted():
    return os.path.exists('/content/drive')

if not is_drive_mounted():
    drive.mount('/content/drive')

def get_notes():
    notes = []
    midi_folder = '/content/drive/MyDrive/LSTMMidi/MIDIs'
    for file in glob.glob(f"{midi_folder}/*.mid"):
        midi = converter.parse(file)
        print(f"Обробка {file}")
        notes_to_parse = midi.flat.notesAndRests
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

def prepare_sequences(notes, n_vocab):
    sequence_length = 100
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    network_output = keras.utils.to_categorical(network_output)
    return (network_input, network_output)

def create_network(network_input, n_vocab):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.LSTM(512, return_sequences=True))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.LSTM(256))
    model.add(keras.layers.Dense(256))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(n_vocab))
    model.add(keras.layers.Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

def train(epochs=50, batch_size=64):
    notes = get_notes()
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)
    model = create_network(network_input, n_vocab)

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

    history = model.fit(network_input, network_output,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_split=0.2,
                        callbacks=[early_stopping, reduce_lr])

    output_folder = '/content/drive/MyDrive/LSTMMidi/MIDI_output'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    model.save(f'{output_folder}/model.h5')
    with open(f'{output_folder}/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return history

def plot_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Втрати на тренуванні')
    plt.plot(history.history['val_loss'], label='Втрати на валідації')
    plt.title('Втрати моделі')
    plt.xlabel('Епоха')
    plt.ylabel('Втрати')
    plt.legend()
    plt.show()

if __name__ == '__main__':
    history = train()
    plot_history(history)
    print("Навчання завершено. Модель і ноти збережено на Google Drive.")

In [ ]:
# @title Налаштування генерації MIDI
# @markdown Введіть бажані параметри для генерації MIDI-файлу.
тривалість_в_хвилинах = 3  # @param {type: "slider", min: 1, max: 10}
фантазія = 2  # @param {type: "slider", min: 0.1, max: 2.0, step: 0.1}
# @markdown ---
# @markdown *Тривалість в хвилинах* обмежує тривалість генерованого MIDI-файлу.
# @markdown *Фантазія (креативність)* контролює непередбачуваність нот (низьке значення - більше передбачуваності, високе - більше креативності).

import pickle
import numpy as np
from music21 import instrument, note, stream, chord
from tensorflow import keras
import os
import random
from google.colab import drive, files
import matplotlib.pyplot as plt

def is_drive_mounted():
    return os.path.exists('/content/drive')

if not is_drive_mounted():
    drive.mount('/content/drive')

def generate(duration_minutes=тривалість_в_хвилинах, temperature=фантазія):
    with open('/content/drive/MyDrive/LSTMMidi/MIDI_output/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = keras.models.load_model('/content/drive/MyDrive/LSTMMidi/MIDI_output/model.h5')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab, duration_minutes, temperature)
    midi_stream = create_midi(prediction_output)

    output_path = '/content/drive/MyDrive/LSTMMidi/MIDI_output/generated_output.mid'
    save_midi_file(midi_stream, output_path)

    print(f"Згенеровано та збережено MIDI файл за адресою: {output_path}")

    visualize_generated_notes(prediction_output)

def prepare_sequences(notes, pitchnames, n_vocab):
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    sequence_length = 100
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
    network_input = np.reshape(network_input, (len(network_input), sequence_length, 1))
    normalized_input = network_input / float(n_vocab)
    return (network_input, normalized_input)

def generate_notes(model, network_input, pitchnames, n_vocab, duration_minutes, temperature):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input[start]
    prediction_output = []
    notes_per_minute = 120
    max_notes = int(duration_minutes * notes_per_minute)

    for note_index in range(max_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction = model.predict(prediction_input, verbose=0)

        prediction = np.log(prediction + 1e-7) / temperature
        prediction = np.exp(prediction) / np.sum(np.exp(prediction))

        if random.random() < 0.8:
            index = np.argmax(prediction)
        else:
            index = np.random.choice(len(prediction[0]), p=prediction[0])

        result = int_to_note[index]
        prediction_output.append(result)
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    return midi_stream

def save_midi_file(midi_stream, file_path):
    """Зберігає згенерований MIDI файл на диск."""
    midi_stream.write('midi', fp=file_path)

def visualize_generated_notes(prediction_output):
    pitches = []

    for pattern in prediction_output:
        if isinstance(pattern, note.Note):
            pitches.append(pattern.pitch.midi)
        elif isinstance(pattern, chord.Chord):
            pitches.append(pattern.root().midi)
        elif '.' in pattern or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            chord_pitches = [int(n) for n in notes_in_chord]
            pitches.append(np.mean(chord_pitches))
        else:
            try:
                note_obj = note.Note(pattern)
                pitches.append(note_obj.pitch.midi)
            except:
                print(f"Не вдалося інтерпретувати шаблон: {pattern}")

    plt.figure(figsize=(12, 4))
    plt.plot(range(len(pitches)), pitches)
    plt.title('Згенеровані ноти')
    plt.xlabel('Послідовність нот')
    plt.ylabel('MIDI Висота')
    plt.show()

if __name__ == '__main__':
    generate()